In [10]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import re
import jieba
import jieba.analyse

#### 1.从数据库导入微博数据

In [11]:
#读取数据
mblog_frame = pd.read_csv('mblog.csv',index_col=None)
mblog_frame.head(2)

,mid,raw_text,isLongText,like_count,comments_count,reposts_count,created_at,source,user_id
0,4415998503823307,中秋福利哈，老爷子支持的转账已就位哈哈<br /><br />关注我的真爱<span cla...,0,919,1003,2817,09-13,NaN,1749268063
1,4416476792619830,"远行随拍，博友假日快乐<span class=""url-icon""><img alt=[微风...",0,73,87,6,16小时前,HUAWEI P30 Pro,1219795431


#### 2.清除text中的非微博正文字符并抽取关键词

In [21]:
# 自定义函数
def clean_text(raw):
    """
    清除text中的非微博正文字符
    返回值类型为元组
    """
    if raw['raw_text']:
        text=re.sub('<[^<]*>','',raw['raw_text']) # 清除多余的html语句
        text=re.sub('[#\n]*','',text) # 清除换行符与#符号
        text=re.sub('(http://.*)$','',text) # 清除文末的网址
        return text 
    else:
        return None
def get_chinese_text(raw):
    """
    清除text中的非中文字符
    只能提取中文字符，微博中的数字以及英文均会丢失
    """
    if raw['text']:
        res_text=''.join(re.findall(r"[\u4e00-\u9fff]{2,}",raw['text']))
        return (raw['mid'],res_text)
    else:
        return None

def get_keywords(raw):
    """
    使用jieba从中文text抽取关键词
    默认抽取20个关键词
    longtext 提取40个关键词
    """
    if raw['chinese_text']:
        if raw['isLongText'] == 1:
            # 当text为长文本时，提取50个关键词
            keywords = jieba.analyse.extract_tags(raw['chinese_text'],topK=50)
        else:
            # 当text为非长文本时，默认提取20个关键词
            keywords = jieba.analyse.extract_tags(raw['chinese_text'])
        return (raw['mid'],keywords)
    else:
        return None

def clean_created_date(raw):
    created_date = raw['created_at']
    if created_date.endswith('前'):
        created_date = '09-15'
    elif created_date.startswith('昨天'):
        created_date = '09-14'
    return created_date

In [15]:
#获取清理后的created_date
mblog_frame['created_date'] = mblog_frame.apply(clean_created_date,axis=1)

In [16]:
# 获取清理后的text
mblog_frame['chinese_text'] = mblog_frame.apply(clean_text,axis=1)

In [17]:
# 以传入字典items()的形式生成DataFrame，指定列名
res_mblog = pd.DataFrame(mblog_frame,columns=['mid','chinese_text','like_count','comments_count','reposts_count','created_date','user_id'])

In [18]:
# 写入csv文件便于查看数据清洗结果
res_mblog.to_csv('clean_mblog.csv', encoding='utf_8_sig',index=False)

In [22]:
# 获取关键字并转换为分散存储的DataFrame
mid_with_keyword = list(mblog_frame.apply(get_keywords,axis=1))
# 这里要把keywords列表存储到数据库，因此需要将keywords列表分开，并与mid对应
keywords_list = [(raw[0],w) for raw in mid_with_keyword for w in raw[1]]
mid_with_keyword = pd.DataFrame(keywords_list,columns=['mid','keyword'])

In [23]:
# 写入csv文件便于查看结果
mid_with_keyword.to_csv('keyword.csv', encoding='utf_8_sig',index=False)

In [10]:
# 将处理结果写入数据库
mid_with_chinese_text.to_sql(name='mblog_text',con=engine,if_exists='append',index=False)
mid_with_keyword.to_sql(name='mblog_keyword',con=engine,if_exists='append',index=False)